In [3]:
import os
import requests
from dotenv import load_dotenv

In [4]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

In [7]:
load_dotenv()

True

In [8]:
@tool
def get_weather(query: str):
    """Get the current weather information for a specified city."""
    print("[debug] get_weather called")

    # Parse latitude and longitude from query
    try:
        lat_lon = query.strip().split(',')
        latitude = float(lat_lon[0].strip())
        longitude = float(lat_lon[1].strip())
        print(f"Latitude: {latitude}, Longitude: {longitude}")
    except:
        # Default to New York if parsing fails
        latitude, longitude = 40.7128, -74.0060

    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m"
    response = requests.get(url)
    data = response.json()
    temperature = data["current"]["temperature_2m"]
    wind_speed = data["current"]["wind_speed_10m"]
    return f"The current temperature is {temperature}°C with a wind speed of {wind_speed} m/s."

    #return Weather(city=query, temperature=temperature, wind_speed=wind_speed)

In [10]:
# Initialize the language model
# Ensure OPENAI_API_KEY environment variable is set
#os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Define the system prompt
SYSTEM_PROMPT = """You are a helpful assistant that can access the weather for any city.
Use the provided tools to answer questions about the weather. Input should be latitude and longitude of the city as two numbers separated by a comma (e.g., '40.7128, -74.0060')."""


In [11]:
agent = create_agent(
    model=llm,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_weather],
)

In [12]:
user_query = "What is the weather in Indianapolis?"
result = agent.invoke({"messages": [HumanMessage(content=user_query)]})

print(f"User Query: {user_query}")
print(f"Agent Response: {result['messages'][-1].content}")


[debug] get_weather called
Latitude: 39.7684, Longitude: -86.1581
User Query: What is the weather in Indianapolis?
Agent Response: The current weather in Indianapolis is -8.2°C with a wind speed of 8.3 m/s.
